# Riak

In [143]:
import riak
from pprintpp import pprint as pp
import json
import pandas as pd

import uuid
import hashlib

## Lectura de información en Pandas

Partimos del dataset normalizado y lo desnormalizamos para guardarlo todo junto en RIAK

In [144]:
df_mov = pd.read_excel("./data/black.xlsx", sheet_name= "Movimientos",  engine='openpyxl')
df_miembros = pd.read_excel("./data/black.xlsx", sheet_name= "Miembros",  engine='openpyxl')
df = pd.merge(df_mov, df_miembros, on = ['id_miembro'], how = 'inner')
df.info()

Exception ignored in: <function RiakClient.__del__ at 0x7f218ad094c0>
Traceback (most recent call last):
  File "/home/learner/venv/lib/python3.8/site-packages/riak/client/__init__.py", line 149, in __del__
    self.close()
  File "/home/learner/venv/lib/python3.8/site-packages/riak/client/__init__.py", line 330, in close
    self._tcp_pool.clear()
  File "/home/learner/venv/lib/python3.8/site-packages/riak/transports/pool.py", line 234, in clear
    for resource in self:
  File "/home/learner/venv/lib/python3.8/site-packages/riak/transports/pool.py", line 290, in __next__
    return self.next()
  File "/home/learner/venv/lib/python3.8/site-packages/riak/transports/pool.py", line 285, in next
    self.__claim_resources()
  File "/home/learner/venv/lib/python3.8/site-packages/riak/transports/pool.py", line 296, in __claim_resources
    self.releaser.wait()
  File "/usr/lib/python3.8/threading.py", line 302, in wait
    waiter.acquire()
KeyboardInterrupt: 


<class 'pandas.core.frame.DataFrame'>
Int64Index: 7624 entries, 0 to 7623
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   id_miembro          7624 non-null   int64         
 1   fecha               7624 non-null   datetime64[ns]
 2   hora                7624 non-null   int64         
 3   minuto              7624 non-null   int64         
 4   importe             7624 non-null   float64       
 5   comercio            6882 non-null   object        
 6   actividad_completa  7621 non-null   object        
 7   actividad           7621 non-null   object        
 8   nombre              7624 non-null   object        
 9   funcion             7624 non-null   object        
 10  organizacion        6060 non-null   object        
dtypes: datetime64[ns](1), float64(1), int64(3), object(6)
memory usage: 714.8+ KB


In [145]:
df.head(5)

,id_miembro,fecha,hora,minuto,importe,comercio,actividad_completa,actividad,nombre,funcion,organizacion
0,77,2006-04-17,11,40,44.56,FCIA TORRES MU¬OZ,FARMACIAS,SALUD,Ricardo Romero de Tejada y Picatoste,concejal,Partido Popular
1,77,2010-12-18,14,35,3.65,PK VELAZQUEZ JORGE JUAN,GARAJES Y APARCAMIENTOS,COCHE,Ricardo Romero de Tejada y Picatoste,concejal,Partido Popular
2,77,2009-04-06,11,56,1000.00,ANTONIO HUESO,"ELECTRODOMESTICOS,EQUIPOS ELECTRICOS",HOGAR,Ricardo Romero de Tejada y Picatoste,concejal,Partido Popular
3,77,2007-10-19,10,49,102.91,CEDIPSA 17017-1 TRUJILLO,CEPSA,COCHE,Ricardo Romero de Tejada y Picatoste,concejal,Partido Popular
4,77,2010-02-11,16,48,68.95,CEDIPSA 17018-1 TRUJILLO,CEPSA,COCHE,Ricardo Romero de Tejada y Picatoste,concejal,Partido Popular


## Funciones de Utilidad

### Funcion para convertir una lista de JSON a Pandas

En la siguiente función partimos del objeto JSON que nos devuelve RIAK y generamos un DataFrame Pandas.

En el caso de la fecha, lo convertimos desde un UNIX Timestamp al formato correcto para Pandas

In [146]:
def json_to_pandas(rows):
    # Convertimos el objeto JSON en un objeto pandas 
    df = pd.read_json(json.dumps(rows))
    
    if 'fecha' in df.columns:
        # Las fechas están en formato UNIX TIMESTAMP. Las volvemos a convertir a formato Date...
        df = df.assign(fecha = pd.to_datetime(df.fecha, unit = 'ms'))

    return df

In [147]:
values = [
    { "column1" : 1, "column2": 2 },
    { "column1" : 13, "column2": 23 }
]

json_to_pandas(values)
    

,column1,column2
0,1,2
1,13,23


### Funcion para obtener el código Hash de un String

La siguiente función convertirá un String a un código HASH que tiene la propiedad de ser siempre el mismo para un determinado String

In [153]:
def hash_string(value):
    value_utf8 = value.encode('utf-8')
    return hashlib.md5(value_utf8).hexdigest()

In [154]:
print(hash_string("Nombre Apellido"))

779955fdd274a9b63005b12a95f8c639


In [155]:
print(hash_string("Nombre Apellido"))

779955fdd274a9b63005b12a95f8c639


In [156]:
print(hash_string("Nombre Otro Apellido"))

21775d9ca6b4e250f0a0f6097e329f37


## Riak ...

In [152]:
# connect to database
cliente = riak.RiakClient()
cliente.ping()

True

Utilizamos un bucket para guardar los movimientos en formato JSON y otro para guardar el importe acumulado por miembro.

En el bucket de movimientos utilizamos una clave autogenerada (no vamos a poder localizar los movimientos por clave) y en el bucket con el importe acumulado la clave es el nombre de la persona en formato HASH, para no tener problemas ni con los espacios del nombre ni con los acentos

Se van a crear los siguientes buckets:

<style>
table {float:left}
</style>


|bucket|Clave|Contenido|
|:-----|-----|---------|
|movimientos|Marca temporal|Todos los datos del dataset en formnato JSON y desnormalizados|
|acum_importes|Código HASH de la persona que realiza el movimiento|**Mapa** con el nombre de la persona y el importe acumulado|

El bucket de movientos tendrá los siguientes índices:

|Nombre del índice|Contenido|¿Que busquedas permite hacer?|
|-|-|-|
|idx_miembro_bin|Nombre de la persona|Localizar movimientos de una persona concreta|

In [109]:
BUCKET_MOVIMIENTOS = 'movimientos'
BUCKET_ACUM_IMPORTES = 'acum_importes'

bucket_mov = cliente.bucket(BUCKET_MOVIMIENTOS)
bucket_acum_importes = cliente.bucket_type('maps').bucket(BUCKET_ACUM_IMPORTES)

Función para eliminar los datos de un BUCKET de Riak

In [110]:
# Se permiten operaciones de listado (No utilizar en PRODUCCION!)
riak.disable_list_exceptions = True

In [111]:
def drop_keys(bucket):
    for keys in bucket.stream_keys():
        for key in keys:
            bucket.delete(key)

In [112]:
# Borramos los datos ..
drop_keys(bucket_mov)
drop_keys(bucket_acum_importes)

In [113]:
riak.disable_list_exceptions = False

## Inserción de información en Riak

Para generar los datos en formato JSON partimos del DataFrame de Pandas y lo exportamos a JSON (en formato String) para luego cargarlo en un diccionario de Python

Observa que cuando se genera el dato en formato JSON, los campos fechas se almacenan como un [TIMESTAMP de unix](http://www.unixtimestamp.com), por lo que habría que volver a convertirlo a fecha según el caso

In [114]:
json_string = df.to_json(orient = 'records')
json_list = json.loads(json_string)

In [115]:
print(len(json_list))

7624


In [116]:
pp(json_list[0])

{
    'actividad': 'SALUD',
    'actividad_completa': 'FARMACIAS',
    'comercio': 'FCIA TORRES MU¬OZ',
    'fecha': 1145232000000,
    'funcion': 'concejal',
    'hora': 11,
    'id_miembro': 77,
    'importe': 44.56,
    'minuto': 40,
    'nombre': 'Ricardo Romero de Tejada y Picatoste',
    'organizacion': 'Partido Popular',
}


Vamos a utilizar el nombre de la persona como clave, por lo que necesitamos convertirla previamente a un código que no lleve ni espacios ni acentos. Para ello utilizaremos la función que convierte un String a un código Hash!

Recorremos el dataset de movimientos, almacenando la información tanto en el bucket de movimientos como en el bucket que acumula los movimientos por cliente.

En este último utilizamos una funcionalidad que nos da Riak para guardar la información tipificada. Utilizamos un mapa que tiene dos elementos: Un registro donde guardamos el nombre del cliente y un contador donde guardamos el importe.

Como el contador no puede guardar datos decimimales, convertimos el importe a un entero multiplicando por 100. Cuando recuperemos la informacion devemos de dividir por 100 para volver a tener decimales

In [119]:
# Bucle de carga de datos ...
for index, movimiento_json in enumerate(json_list):
    key = uuid.uuid4().hex
    hash_nombre =  hash_string(movimiento_json["nombre"])
    
    movimiento = bucket_mov.new(key, movimiento_json)
    movimiento.add_index('idx_miembro_bin', hash_nombre)
    movimiento.store()
    
    map_nombre = bucket_acum_importes.new(hash_nombre)
    map_nombre.registers['nombre'].assign(movimiento_json["nombre"])
    map_nombre.counters['importe'].increment(int(round(movimiento_json["importe"] * 100)))
    map_nombre.store()
    
    if index % 500 == 0:
        print(index)

0
500
1000
1500
2000
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!
Yes!

Comprobamos que los datos están correctos en los 3 primeros registros

In [130]:
riak.disable_list_exceptions = True

values = []
for key in bucket_mov.get_keys()[0:3]:
    data = bucket_mov.get(key).data
    values.append(data)
    
riak.disable_list_exceptions = False

json_to_pandas(values)

,id_miembro,fecha,hora,minuto,importe,comercio,actividad_completa,actividad,nombre,funcion,organizacion
0,29,2007-11-26,19,27,74.00,"BOCH, SA",CONFECCION TEXTIL EN GENERAL,ROPA,Javier de Miguel Sánchez,concejal,Partido Popular
1,78,2008-07-27,16,45,30.04,LUCIO IBA#EZ GRANDE,TAXIS,COCHE,Rodolfo Benito Valenciano,concejal,CC OO
2,73,2006-06-20,16,37,85.05,RESTAURANTE EL CEBADERO,RESTAURANTES RESTO,RESTAURANTE,Ramón Espinar Gallego,concejal,PSOE


Ahora mostramos los datos del bucket con los datos acumulados ...

In [131]:
riak.disable_list_exceptions = True

for key in bucket_acum_importes.get_keys()[0:3]:
    data = bucket_acum_importes.get(key).value
    print('%s -> %s' % (key, data))

riak.disable_list_exceptions = False

dc08b13bdd86444d8ded7273a8e3ff90 -> {('importe', 'counter'): 1157991, ('nombre', 'register'): 'Jorge Gómez Moreno'}
929daca40203a3aa05b3446ecfe7ef46 -> {('importe', 'counter'): 699703, ('nombre', 'register'): 'Beltrán Gutiérrez Moliner'}
d51e4e145b8fb8b841381c565704890e -> {('importe', 'counter'): 3155645, ('nombre', 'register'): 'Carmen Contreras Gómez'}


# Lectura de información en Riak

### Lectura del bucket de movimientos

Cargamos todos los movimientos de la base de datos y lo guardamos en un objeto Pandas, ya que hay ciertas preguntas que no pueden resolverse directamente por esta base de datos

Recuerda que RIAK permite obtener información de un clave, pero no le es posible devolver la información ordenada ...

In [132]:
rows = []
riak.disable_list_exceptions = True
for keys in bucket_mov.stream_keys():
    for key in keys:
        # print('Key %s' % key )
        rows.append(bucket_mov.get(key).data)

riak.disable_list_exceptions = False        
# Convertimos el objeto json en un objeto pandas 
df_movimientos = json_to_pandas(rows)

In [133]:
df_movimientos.head()

,id_miembro,fecha,hora,minuto,importe,comercio,actividad_completa,actividad,nombre,funcion,organizacion
0,17,2008-03-29,15,51,600.00,"ADOC GOURMET, S.A.",RESTAURANTES RESTO,RESTAURANTE,Francisco Baquero Noriega,concejal,CC OO
1,12,2009-08-24,19,8,87.50,EL CORTE INGLES,HIPERCOR SUPERMERCADOS EL CORTE INGLES,SUPERMERCADO,Cándido Cerón Escudero,concejal,Partido Popular
2,40,2006-04-27,0,24,56.15,PUMMAROLA,RESTAURANTES RESTO,RESTAURANTE,José María Arteta Vico,concejal,PSOE
3,49,2010-04-22,11,32,44.81,LIBRERIA LE,"LIBRERIAS, PAPELERIAS Y DISCOS",LIBRERIA,Juan José Azcona Olondriz,concejal,CC OO
4,37,2003-12-22,14,24,29.55,HIPERCOR CAMPO NACIONES-SUPER,HIPERCOR SUPERMERCADOS EL CORTE INGLES,SUPERMERCADO,José Carlos Contreras Gómez,directivo,None


### Los 10 movimientos mas caros

Esta consulta no la podemos contestar directamente con la Base de Datos por lo que nos apoyamos en un proceso en el cliente

In [134]:
df_movimientos \
    .sort_values('importe', ascending=False)\
    .filter(['nombre', 'fecha', 'actividad_completa', 'importe']) \
    .head(10)

,nombre,fecha,actividad_completa,importe
7324,Enrique de la Torre Martínez,2007-11-28,AGENCIAS BANCARIAS(ANTICIPO VENTANILLA),12000.00
4211,Ramón Ferraz Ricarte,2007-12-19,AGENCIAS BANCARIAS(ANTICIPO VENTANILLA),9424.08
3586,Enrique de la Torre Martínez,2008-10-23,AGENCIAS BANCARIAS(ANTICIPO VENTANILLA),9000.00
6769,Enrique de la Torre Martínez,2006-11-29,AGENCIAS BANCARIAS(ANTICIPO VENTANILLA),8000.00
486,Miguel Blesa de la Parra,2006-12-26,EL CORTE INGLES,6397.31
1616,Rafael Spottorno Díaz Caro,2007-11-30,CONFECCION TEXTIL EN GENERAL,6375.00
2048,Carmen Contreras Gómez,2006-08-13,"HOTELES 4 Y 5 ESTRELLAS,BALNEARIOS,CAMPI",6198.02
2665,Matías Amat Roca,2006-08-13,BRITISH AIRWAYS - BRITISH A,5763.10
3192,Juan Manuel Astorqui Portera,2005-07-31,AUTOM.Y MOTOCICLETAS ( VENTAS Y REPARAC),5403.97
530,Enrique de la Torre Martínez,2004-12-06,AGENCIAS BANCARIAS(ANTICIPO VENTANILLA),5390.00


### Los movimientos de una persona concreta

Para obtener los movimientos de una persona en concreto si podemos **utilizar el índice** que habíamos creado Ad-hoc, aunque la ordenación se realiza en Pandas

In [137]:
rows = []
keys = bucket_mov.stream_index("idx_miembro_bin", hash_string(u"Ildefonso José Sánchez Barcoj"))
for keys in keys.results:
    for movimiento_key in keys:
        rows.append(bucket_mov.get(movimiento_key).data)
    
df = json_to_pandas(rows)

In [138]:
df \
    .sort_values('importe', ascending=False) \
    .filter(['nombre', 'fecha', 'actividad_completa', 'importe']) \
    .head(10)

,nombre,fecha,actividad_completa,importe
163,Ildefonso José Sánchez Barcoj,2009-05-04,AGENCIAS BANCARIAS(ANTICIPO VENTANILLA),1500.00
114,Ildefonso José Sánchez Barcoj,2010-07-07,EL CORTE INGLES,1478.07
56,Ildefonso José Sánchez Barcoj,2008-03-21,"HOTELES 4 Y 5 ESTRELLAS,BALNEARIOS,CAMPI",1392.79
63,Ildefonso José Sánchez Barcoj,2005-10-28,LOUIS VUITTON/LOEWE,1375.00
224,Ildefonso José Sánchez Barcoj,2009-09-28,AGENCIAS BANCARIAS(ANTICIPO VENTANILLA),1000.00
35,Ildefonso José Sánchez Barcoj,2012-03-15,IBERIA,940.00
26,Ildefonso José Sánchez Barcoj,2010-02-05,CONFECCION TEXTIL EN GENERAL,930.00
144,Ildefonso José Sánchez Barcoj,2005-12-19,EL CORTE INGLES,731.00
260,Ildefonso José Sánchez Barcoj,2009-03-20,PARADORES NACIONALES,708.11
151,Ildefonso José Sánchez Barcoj,2009-12-07,"HOTELES 4 Y 5 ESTRELLAS,BALNEARIOS,CAMPI",672.44


### Las 10 personas que mas han gastado

Hemos creado un bucket especifico con información agregada para poder contestar a esta pregunta. Como la información está en un **mapa** la tenemos que convertir a JSON para poder visualizarla con Pandas

In [141]:
rows = []
riak.disable_list_exceptions = True
for keys in bucket_acum_importes.stream_keys():
    #print(keys)
    for key in keys:
        map_nombre = bucket_acum_importes.get(key)
        rows.append({'nombre' : map_nombre.registers['nombre'].value,
                     "importe" : float(map_nombre.counters['importe'].value) / 100})
        
riak.disable_list_exceptions = False

# Convertimos el objeto json en un objeto pandas 
df = json_to_pandas(rows)

Utilizamos Pandas para ordenar la información ..,

In [142]:
df.sort_values('importe', ascending=False).head(10)

,nombre,importe
78,Enrique de la Torre Martínez,58826.28
65,José Antonio Moral Santín,52447.07
17,Ramón Ferraz Ricarte,45371.52
3,Miguel Blesa de la Parra,40298.84
33,Ildefonso José Sánchez Barcoj,39437.99
18,Juan Manuel Astorqui Portera,36627.89
27,Francisco Baquero Noriega,35256.67
53,Matías Amat Roca,33434.74
0,Carmen Contreras Gómez,31556.45
62,Estanislao Rodríguez-Ponga Salamanca,29350.65
